In [6]:
import os
import sys
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

from torchvision.transforms import transforms

import utils.datasets as datasets
from utils.logger import Logger
from utils.meter import AverageMeter, ProgressMeter
from utils.estimation import estimate
from utils.accuracy import accuracy
from utils.rank import accuracy as rank
from utils.mAP import accuracy as mAP

from nets.resnet50 import Bottleneck, ResNet50
from nets.head import ProjHead

from center_loss import CenterLoss

In [7]:
def train(model, criterion, optimizer, trainLoader, testLoader, num_classes, epoch, use_gpu):
    model.train()
    
    CrossEntropyLosses = AverageMeter("CrossEntropyLoss", ":6.4f")
    CenterLosses = AverageMeter("CenterLoss", ":6.4f")
    losses = AverageMeter("Loss", ":6.4f")
    top1 = AverageMeter("ceAcc@1", ":6.2f")
    top5 = AverageMeter("ceAcc@5", ":6.2f")
    top10 = AverageMeter("ceAcc@10", ":6.2f")
    progress = ProgressMeter(
        len(trainLoader),
        [CrossEntropyLosses, CenterLosses, losses, top1, top5, top10],
        prefix="Epoch: [{}]".format(epoch+1),
    )
    
    print_frequency = 6
    
    for i, (inputs, labels) in enumerate(trainLoader):
        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
        
        outputs = model(inputs)
        
        out, loss, celoss, centerloss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        for param in criterion.parameters():
            param.grad.data *= (1. /criterion.lamda)
        optimizer.step()        
        
        acc1, acc5, acc10 = accuracy(out, labels, topk=(1,5,10))
        top1.update(acc1[0], labels.shape[0])
        top5.update(acc5[0], labels.shape[0])
        top10.update(acc10[0], labels.shape[0])
        CrossEntropyLosses.update(celoss.item(), labels.shape[0])
        CenterLosses.update(centerloss.item(), labels.shape[0])
        losses.update(loss.item(), labels.shape[0])
        
        if i % print_frequency == 0:
            progress.display(i+1)


In [8]:
@torch.no_grad()
def test(model, testLoader):

    dataset_dir = '/home/tianyi/WORK/CUHK01/dataset'
    Batch_Size = 128
    num_pos = 4
    feat_dim = 2048
    
    MemoryBank = torch.zeros([1, feat_dim])
    labels = torch.IntTensor([1])

    MemoryBank = MemoryBank.cuda()
    labels = labels.cuda()
        
    for i, (inputs, labels_i) in enumerate(testLoader):
        inputs = inputs.cuda()
        labels_i = labels_i.cuda()
            
        outputs = model(inputs)
        MemoryBank = torch.cat( (MemoryBank, outputs), dim=0)
        labels = torch.cat( (labels, labels_i), dim=0)
        
    x = MemoryBank[1:]
    labels = labels[1:]
    N = labels.shape[0]
    
    x2 = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(N, N)
    distmat = x2 + x2.t()
    distmat.addmm_(1, -2, x, x.t())
    
    acc1_rank, acc5_rank, acc10_rank = rank(distmat, labels, topk=(1, 5, 10))
    
    print("\nTest (Rank):\nACC@1: {0:10.4f}%    ACC@5: {1:10.4f}%    ACC@10: {2:10.4f}%\n\n".format(acc1_rank[0], acc5_rank[0], acc10_rank[0]))

In [9]:
def main():
    log_dir = 'logs'
    dataset_dir = '/home/tianyi/WORK/CUHK01/dataset'
    dataset_name = 'CUHK01'
    num_pos = 4
    timestamp = time.strftime("%Y-%m-%d-%H-%M")
    log_dir = os.path.join(log_dir, dataset_name, timestamp)
    weights_dir = os.path.join(log_dir, 'weights')
    os.makedirs(weights_dir)
    centers_dir = os.path.join(log_dir, 'centers')
    os.makedirs(centers_dir)
    
    seed = 1
    use_gpu = True
    
    __console__ = sys.stdout
    log_file = Logger(os.path.join(log_dir, 'logfile.txt'))
    sys.stdout = log_file
    
    torch.manual_seed(seed)
    
    if use_gpu:
        device = "cuda"
        cudnn.benchmark = True
        torch.cuda.manual_seed_all(seed)
        print('Device: GPU')
    else:
        device = "cpu"
        print('Device: CPU')

    Batch_Size = 128
    
    LR_model = 1e-3
    LR_centerLoss = 1e-2
    
    MaxEpoch = 300
    
    transform = transforms.Compose(
        [transforms.ToTensor(), 
         transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), 
         transforms.Resize((224,224))]
    )
    train_dataset = datasets.CUHK01(root=dataset_dir, train=True, transform=transform)
    test_dataset = datasets.CUHK01(root=dataset_dir, train=False, transform=transform)
    
    num_classes = int(len(train_dataset)//num_pos)
    
    trainLoader = DataLoader(
        train_dataset,
        batch_size = Batch_Size,
        shuffle = True
    )

    testLoader = DataLoader(
        test_dataset,
        batch_size = Batch_Size,
        shuffle = False
    )
    
    feat_dim = 2048
    
    res50 = ResNet50(Bottleneck).to(device)
    model = res50
    
    criterion = CenterLoss(feat_dim=feat_dim, num_classes=num_classes, use_gpu=True)
    
    optimizer = optim.Adam([
        {'params': model.parameters(), 'lr': LR_model},
        {'params': criterion.parameters(), 'lr': LR_centerLoss},
    ])
    
    for epoch in range(MaxEpoch):
        print(134*'=' + "\n\nEpoch {}/{}".format(epoch+1, MaxEpoch))
        
        train(model, criterion, optimizer, trainLoader, testLoader, num_classes, epoch, use_gpu=True)
        
        test(model, testLoader)
        
        torch.save(model.state_dict(), os.path.join(weights_dir, '{}.pth.tar'.format(epoch+1)))
        torch.save(criterion.centers, os.path.join(centers_dir, '{}.pkl'.format(epoch+1)))
        
    sys.stdout = __console__

In [10]:
main()

Device: GPU

Epoch 1/300
Epoch: [1][ 1/28]	CrossEntropyLoss 6.8385 (6.8385)	CenterLoss 1209.7883 (1209.7883)	Loss 127.8173 (127.8173)	ceAcc@1   0.00 (  0.00)	ceAcc@5   0.78 (  0.78)	ceAcc@10   0.78 (  0.78)
Epoch: [1][ 7/28]	CrossEntropyLoss 7.4098 (8.5488)	CenterLoss 1009.0308 (1070.7416)	Loss 108.3128 (115.6230)	ceAcc@1   0.00 (  0.11)	ceAcc@5   0.00 (  0.45)	ceAcc@10   0.00 (  0.89)
Epoch: [1][13/28]	CrossEntropyLoss 7.0481 (8.1053)	CenterLoss 989.8195 (1035.0101)	Loss 106.0301 (111.6063)	ceAcc@1   0.00 (  0.06)	ceAcc@5   0.00 (  0.36)	ceAcc@10   0.00 (  0.66)
Epoch: [1][19/28]	CrossEntropyLoss 7.3297 (7.7770)	CenterLoss 962.6489 (1012.3803)	Loss 103.5945 (109.0151)	ceAcc@1   0.00 (  0.04)	ceAcc@5   0.00 (  0.25)	ceAcc@10   0.00 (  0.45)
Epoch: [1][25/28]	CrossEntropyLoss 6.8831 (7.5829)	CenterLoss 926.5857 (994.3680)	Loss 99.5416 (107.0197)	ceAcc@1   0.00 (  0.03)	ceAcc@5   0.00 (  0.22)	ceAcc@10   0.00 (  0.38)

Test (Rank):
ACC@1:    10.5263%    ACC@5:    28.0702%    ACC@10:    4

Epoch: [8][13/28]	CrossEntropyLoss 6.6507 (6.6727)	CenterLoss 329.6246 (340.9390)	Loss 39.6131 (40.7666)	ceAcc@1   0.00 (  0.12)	ceAcc@5   0.78 (  0.78)	ceAcc@10   0.78 (  1.86)
Epoch: [8][19/28]	CrossEntropyLoss 6.6269 (6.6676)	CenterLoss 326.6315 (336.7311)	Loss 39.2901 (40.3408)	ceAcc@1   0.00 (  0.21)	ceAcc@5   0.78 (  0.82)	ceAcc@10   2.34 (  1.81)
Epoch: [8][25/28]	CrossEntropyLoss 6.7217 (6.6753)	CenterLoss 314.5649 (332.4202)	Loss 38.1782 (39.9173)	ceAcc@1   0.00 (  0.19)	ceAcc@5   0.00 (  0.78)	ceAcc@10   0.00 (  1.72)

Test (Rank):
ACC@1:     9.2732%    ACC@5:    23.8095%    ACC@10:    38.0952%



Epoch 9/300
Epoch: [9][ 1/28]	CrossEntropyLoss 6.6818 (6.6818)	CenterLoss 301.0966 (301.0966)	Loss 36.7915 (36.7915)	ceAcc@1   0.00 (  0.00)	ceAcc@5   0.00 (  0.00)	ceAcc@10   0.78 (  0.78)
Epoch: [9][ 7/28]	CrossEntropyLoss 6.6619 (6.6743)	CenterLoss 294.6267 (297.5394)	Loss 36.1246 (36.4282)	ceAcc@1   0.00 (  0.22)	ceAcc@5   0.78 (  0.78)	ceAcc@10   2.34 (  1.56)
Epoch: [9][13/28]

Epoch: [15][25/28]	CrossEntropyLoss 6.5338 (6.5564)	CenterLoss 111.0969 (118.1113)	Loss 17.6435 (18.3675)	ceAcc@1   0.00 (  0.31)	ceAcc@5   2.34 (  1.69)	ceAcc@10   3.91 (  2.84)

Test (Rank):
ACC@1:    10.7769%    ACC@5:    26.5664%    ACC@10:    39.3484%



Epoch 16/300
Epoch: [16][ 1/28]	CrossEntropyLoss 6.3541 (6.3541)	CenterLoss 110.0009 (110.0009)	Loss 17.3542 (17.3542)	ceAcc@1   0.00 (  0.00)	ceAcc@5   3.12 (  3.12)	ceAcc@10   8.59 (  8.59)
Epoch: [16][ 7/28]	CrossEntropyLoss 6.5197 (6.4523)	CenterLoss 105.6784 (106.3729)	Loss 17.0876 (17.0896)	ceAcc@1   0.78 (  0.56)	ceAcc@5   0.78 (  2.46)	ceAcc@10   0.78 (  4.69)
Epoch: [16][13/28]	CrossEntropyLoss 6.4675 (6.4588)	CenterLoss 99.9570 (104.7391)	Loss 16.4632 (16.9327)	ceAcc@1   0.00 (  0.60)	ceAcc@5   2.34 (  2.34)	ceAcc@10   3.91 (  4.45)
Epoch: [16][19/28]	CrossEntropyLoss 6.6383 (6.4687)	CenterLoss 103.9061 (103.5457)	Loss 17.0289 (16.8233)	ceAcc@1   0.00 (  0.53)	ceAcc@5   0.78 (  2.01)	ceAcc@10   1.56 (  4.24)
Epoch: [16][

Epoch: [23][ 1/28]	CrossEntropyLoss 6.1458 (6.1458)	CenterLoss 38.1012 (38.1012)	Loss 9.9559 (9.9559)	ceAcc@1   2.34 (  2.34)	ceAcc@5   6.25 (  6.25)	ceAcc@10  12.50 ( 12.50)
Epoch: [23][ 7/28]	CrossEntropyLoss 6.0336 (6.1117)	CenterLoss 36.9349 (38.5341)	Loss 9.7271 (9.9651)	ceAcc@1   1.56 (  1.67)	ceAcc@5   4.69 (  6.03)	ceAcc@10   9.38 ( 10.49)
Epoch: [23][13/28]	CrossEntropyLoss 6.0095 (6.1309)	CenterLoss 36.8525 (38.2148)	Loss 9.6948 (9.9524)	ceAcc@1   2.34 (  1.32)	ceAcc@5   7.03 (  5.17)	ceAcc@10  10.16 (  8.71)
Epoch: [23][19/28]	CrossEntropyLoss 6.1560 (6.1303)	CenterLoss 35.0524 (37.3523)	Loss 9.6613 (9.8655)	ceAcc@1   0.00 (  1.07)	ceAcc@5   1.56 (  4.56)	ceAcc@10   6.25 (  8.10)
Epoch: [23][25/28]	CrossEntropyLoss 5.9979 (6.1170)	CenterLoss 34.9261 (36.8302)	Loss 9.4905 (9.8001)	ceAcc@1   0.00 (  0.94)	ceAcc@5   3.12 (  4.31)	ceAcc@10   6.25 (  7.81)

Test (Rank):
ACC@1:    17.2932%    ACC@5:    33.8346%    ACC@10:    49.6241%



Epoch 24/300
Epoch: [24][ 1/28]	CrossEntropy

Epoch: [30][19/28]	CrossEntropyLoss 5.5791 (5.4752)	CenterLoss 12.5843 (13.5490)	Loss 6.8375 (6.8300)	ceAcc@1   0.78 (  4.19)	ceAcc@5   6.25 ( 12.54)	ceAcc@10  13.28 ( 18.63)
Epoch: [30][25/28]	CrossEntropyLoss 5.4856 (5.4919)	CenterLoss 12.0512 (13.3527)	Loss 6.6907 (6.8271)	ceAcc@1   1.56 (  3.72)	ceAcc@5  11.72 ( 12.00)	ceAcc@10  19.53 ( 17.94)

Test (Rank):
ACC@1:    25.0627%    ACC@5:    47.3684%    ACC@10:    63.9098%



Epoch 31/300
Epoch: [31][ 1/28]	CrossEntropyLoss 5.3713 (5.3713)	CenterLoss 11.7804 (11.7804)	Loss 6.5494 (6.5494)	ceAcc@1   5.47 (  5.47)	ceAcc@5  17.19 ( 17.19)	ceAcc@10  25.00 ( 25.00)
Epoch: [31][ 7/28]	CrossEntropyLoss 5.5406 (5.2510)	CenterLoss 11.8495 (11.8902)	Loss 6.7255 (6.4401)	ceAcc@1   0.78 (  5.36)	ceAcc@5   8.59 ( 18.08)	ceAcc@10  17.97 ( 25.45)
Epoch: [31][13/28]	CrossEntropyLoss 5.2212 (5.3308)	CenterLoss 11.1487 (11.8096)	Loss 6.3360 (6.5118)	ceAcc@1   8.59 (  4.81)	ceAcc@5  21.88 ( 15.63)	ceAcc@10  25.00 ( 22.54)
Epoch: [31][19/28]	CrossEntropy

...


Test (Rank):
ACC@1:    66.1654%    ACC@5:    87.7193%    ACC@10:    91.4787%



Epoch 264/300
Epoch: [264][ 1/28]	CrossEntropyLoss 0.0062 (0.0062)	CenterLoss 0.3520 (0.3520)	Loss 0.0414 (0.0414)	ceAcc@1 100.00 (100.00)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [264][ 7/28]	CrossEntropyLoss 0.0043 (0.0069)	CenterLoss 0.3822 (0.3511)	Loss 0.0425 (0.0420)	ceAcc@1 100.00 ( 99.89)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [264][13/28]	CrossEntropyLoss 0.0053 (0.0062)	CenterLoss 0.4993 (0.3903)	Loss 0.0552 (0.0452)	ceAcc@1 100.00 ( 99.94)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [264][19/28]	CrossEntropyLoss 0.0053 (0.0062)	CenterLoss 0.4292 (0.4108)	Loss 0.0482 (0.0472)	ceAcc@1 100.00 ( 99.96)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [264][25/28]	CrossEntropyLoss 0.0066 (0.0065)	CenterLoss 0.4310 (0.4156)	Loss 0.0497 (0.0481)	ceAcc@1 100.00 ( 99.97)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)

Test (Rank):
ACC@1:    67.9198%   

Epoch: [271][13/28]	CrossEntropyLoss 0.0116 (0.0065)	CenterLoss 0.4888 (0.3999)	Loss 0.0605 (0.0465)	ceAcc@1  99.22 ( 99.94)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [271][19/28]	CrossEntropyLoss 0.0059 (0.0060)	CenterLoss 0.4287 (0.4198)	Loss 0.0488 (0.0480)	ceAcc@1 100.00 ( 99.96)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [271][25/28]	CrossEntropyLoss 0.0133 (0.0070)	CenterLoss 0.4274 (0.4233)	Loss 0.0560 (0.0493)	ceAcc@1 100.00 ( 99.94)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)

Test (Rank):
ACC@1:    64.6617%    ACC@5:    86.7168%    ACC@10:    90.7268%



Epoch 272/300
Epoch: [272][ 1/28]	CrossEntropyLoss 0.0046 (0.0046)	CenterLoss 0.3201 (0.3201)	Loss 0.0366 (0.0366)	ceAcc@1 100.00 (100.00)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [272][ 7/28]	CrossEntropyLoss 0.0041 (0.0063)	CenterLoss 0.3686 (0.3506)	Loss 0.0410 (0.0413)	ceAcc@1 100.00 ( 99.89)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [272][13/28]	CrossEntropyLos


Test (Rank):
ACC@1:    66.4160%    ACC@5:    86.2155%    ACC@10:    90.7268%



Epoch 279/300
Epoch: [279][ 1/28]	CrossEntropyLoss 0.0049 (0.0049)	CenterLoss 0.2263 (0.2263)	Loss 0.0276 (0.0276)	ceAcc@1 100.00 (100.00)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [279][ 7/28]	CrossEntropyLoss 0.0038 (0.0050)	CenterLoss 0.3524 (0.2954)	Loss 0.0391 (0.0346)	ceAcc@1 100.00 (100.00)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [279][13/28]	CrossEntropyLoss 0.0031 (0.0045)	CenterLoss 0.3380 (0.3157)	Loss 0.0369 (0.0361)	ceAcc@1 100.00 (100.00)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [279][19/28]	CrossEntropyLoss 0.0166 (0.0052)	CenterLoss 0.3918 (0.3424)	Loss 0.0557 (0.0394)	ceAcc@1  99.22 ( 99.96)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [279][25/28]	CrossEntropyLoss 0.0051 (0.0057)	CenterLoss 0.4026 (0.3621)	Loss 0.0454 (0.0419)	ceAcc@1 100.00 ( 99.94)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)

Test (Rank):
ACC@1:    66.6667%   

Epoch: [286][13/28]	CrossEntropyLoss 0.0043 (0.0068)	CenterLoss 0.4694 (0.3529)	Loss 0.0512 (0.0421)	ceAcc@1 100.00 ( 99.82)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [286][19/28]	CrossEntropyLoss 0.0041 (0.0062)	CenterLoss 0.4160 (0.3671)	Loss 0.0457 (0.0430)	ceAcc@1 100.00 ( 99.88)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [286][25/28]	CrossEntropyLoss 0.0038 (0.0060)	CenterLoss 0.3984 (0.3827)	Loss 0.0436 (0.0443)	ceAcc@1 100.00 ( 99.91)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)

Test (Rank):
ACC@1:    65.9148%    ACC@5:    86.9674%    ACC@10:    91.2281%



Epoch 287/300
Epoch: [287][ 1/28]	CrossEntropyLoss 0.0026 (0.0026)	CenterLoss 0.2415 (0.2415)	Loss 0.0268 (0.0268)	ceAcc@1 100.00 (100.00)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [287][ 7/28]	CrossEntropyLoss 0.0334 (0.0263)	CenterLoss 0.5059 (0.3896)	Loss 0.0840 (0.0652)	ceAcc@1  98.44 ( 99.33)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [287][13/28]	CrossEntropyLos


Test (Rank):
ACC@1:    68.6717%    ACC@5:    87.2180%    ACC@10:    91.2281%



Epoch 294/300
Epoch: [294][ 1/28]	CrossEntropyLoss 0.0050 (0.0050)	CenterLoss 0.2761 (0.2761)	Loss 0.0326 (0.0326)	ceAcc@1 100.00 (100.00)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [294][ 7/28]	CrossEntropyLoss 0.0047 (0.0068)	CenterLoss 0.3465 (0.3205)	Loss 0.0393 (0.0388)	ceAcc@1 100.00 ( 99.78)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [294][13/28]	CrossEntropyLoss 0.0037 (0.0059)	CenterLoss 0.4203 (0.3797)	Loss 0.0457 (0.0438)	ceAcc@1 100.00 ( 99.88)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [294][19/28]	CrossEntropyLoss 0.0045 (0.0052)	CenterLoss 0.4535 (0.4073)	Loss 0.0498 (0.0459)	ceAcc@1 100.00 ( 99.92)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)
Epoch: [294][25/28]	CrossEntropyLoss 0.0078 (0.0055)	CenterLoss 0.4428 (0.4116)	Loss 0.0520 (0.0467)	ceAcc@1 100.00 ( 99.91)	ceAcc@5 100.00 (100.00)	ceAcc@10 100.00 (100.00)

Test (Rank):
ACC@1:    68.9223%   